# Capstone Phase 1: AI-Powered Requirements & PRD Generation

**Project:** StaffAlloc - AI-Powered Project Staffing & Hours Allocation Tool

**Objective:** Use Large Language Models (LLMs) to transform our project idea into structured requirements, user stories, and a comprehensive Product Requirements Document (PRD).

**Estimated Time:** 90 minutes

---

## 📋 Before You Begin

**Prerequisites:**
1. ✅ Create a `.env` file in the project root with your `GOOGLE_API_KEY`
2. ✅ Get your API key from: https://aistudio.google.com/app/apikey

**Format of `.env` file:**
```
GOOGLE_API_KEY=your_actual_api_key_here
```

---

## 📖 Introduction

This notebook combines the workflows from Day 1 Labs 1 and 2 to complete Phase 1 of our capstone project. We'll start with our StaffAlloc idea and use AI to generate:
1. Features and user personas
2. Structured user stories with acceptance criteria in JSON format
3. A formal Product Requirements Document (PRD)
4. A Pydantic validation model

All artifacts will be saved to the `Artifacts/Documentation/` folder.

**How to use this notebook:**
- Run **Step 0** first to install required packages
- Then run each cell in order from top to bottom


## Step 0: Install Required Packages

**Run this cell FIRST** to ensure all required packages are installed in the notebook's kernel environment.

This will install packages directly into the Python environment that Jupyter is using.


In [1]:
# Install required packages in the current kernel environment
import sys
import subprocess

print("Installing required packages in the current kernel environment...")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print("=" * 70)

# List of required packages for Phase 1
required_packages = [
    'python-dotenv',
    'google-genai',
    'pydantic',
    'ipython',
]

# Install each package
for package in required_packages:
    print(f"\n📦 Installing {package}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"   ✓ {package} installed successfully")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Failed to install {package}: {e}")

print("\n" + "=" * 70)
print("✅ Package installation complete!")
print("\nYou can now proceed to Step 1.")
print("=" * 70)


Installing required packages in the current kernel environment...
Python executable: c:\Users\640109\T1Capstone\.venv\Scripts\python.exe
Python version: 3.13.9 (tags/v3.13.9:8183fa5, Oct 14 2025, 14:09:13) [MSC v.1944 64 bit (AMD64)]

📦 Installing python-dotenv...
   ✓ python-dotenv installed successfully

📦 Installing google-genai...
   ✓ google-genai installed successfully

📦 Installing pydantic...
   ✓ pydantic installed successfully

📦 Installing ipython...
   ✓ ipython installed successfully

✅ Package installation complete!

You can now proceed to Step 1.


## Step 1: Setup

This cell performs the following setup tasks:

1. **Locate Project Root**: Find the project directory and add it to Python's path
2. **Load Environment Variables**: Load API keys from the `.env` file
3. **Verify Dependencies**: Check that required packages are installed
4. **Initialize LLM Client**: Set up the Google Gemini AI client

**Before running this cell, make sure:**
- ✅ You have installed dependencies: `pip install -r requirements.txt`
- ✅ You have created a `.env` file with your `GOOGLE_API_KEY`
- ✅ Your API key is from: https://aistudio.google.com/app/apikey

**Expected output:** You should see checkmarks (✓) for each step and a final success message.


In [4]:
import sys
import os
import json

# ============================================================================
# Step 1a: Find Project Root and Add to Path
# ============================================================================
# Get the project root directory (one level up from Python Notebooks)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"📁 Project root: {project_root}")

# Add project root to Python path so we can import utils
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# ============================================================================
# Step 1b: Load Environment Variables from .env
# ============================================================================
# Check if required packages are installed
try:
    from dotenv import load_dotenv
    print("✓ python-dotenv is installed")
except ImportError:
    print("❌ ERROR: python-dotenv is not installed!")
    print("   Run: pip install python-dotenv")
    raise

# Load the .env file from project root
env_path = os.path.join(project_root, '.env')
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"✓ Loaded .env file from: {env_path}")
else:
    print(f"⚠️  WARNING: .env file not found at: {env_path}")
    print("   Please create a .env file with your API keys.")
    print("   See SETUP.md for instructions.")

# Verify API key is loaded
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    # Only show first/last few characters for security
    masked_key = f"{google_api_key[:8]}...{google_api_key[-4:]}" if len(google_api_key) > 12 else "***"
    print(f"✓ GOOGLE_API_KEY found: {masked_key}")
else:
    print("❌ ERROR: GOOGLE_API_KEY not found in environment!")
    print("   Please add GOOGLE_API_KEY to your .env file")
    print("   Get your key at: https://aistudio.google.com/app/apikey")

# ============================================================================
# Step 1c: Import Utils and Initialize LLM Client
# ============================================================================
try:
    from utils import setup_llm_client, get_completion, save_artifact
    print("✓ Utils imported successfully")
except ImportError as e:
    print(f"❌ ERROR importing utils: {e}")
    raise

# Check for google-genai package
try:
    import google.genai
    print("✓ google-genai package is installed")
except ImportError:
    print("❌ ERROR: google-genai is not installed!")
    print("   Run: pip install google-genai")
    raise

# ============================================================================
# Step 1d: Initialize the LLM Client
# ============================================================================
print("\n" + "="*60)
print("Initializing LLM Client...")
print("="*60)

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

if client is not None and api_provider is not None:
    print(f"\n✅ SUCCESS! LLM Client initialized")
    print(f"   Provider: {api_provider}")
    print(f"   Model: {model_name}")
else:
    print("\n❌ FAILED to initialize LLM client!")
    print("   Please check the error messages above.")
    print("\nCommon issues:")
    print("   1. Missing dependencies: Run 'pip install -r requirements.txt'")
    print("   2. Missing .env file: Create .env with GOOGLE_API_KEY")
    print("   3. Invalid API key: Check your key at https://aistudio.google.com/app/apikey")
    raise RuntimeError("LLM client initialization failed")


📁 Project root: c:\Users\640109\T1Capstone
✓ python-dotenv is installed
✓ Loaded .env file from: c:\Users\640109\T1Capstone\.env
✓ GOOGLE_API_KEY found: AIzaSyBZ..._C7Q
✓ Utils imported successfully
✓ google-genai package is installed

Initializing LLM Client...


2025-11-05 14:07:17,422 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None



✅ SUCCESS! LLM Client initialized
   Provider: google
   Model: gemini-2.5-pro


## Step 2: Define the Problem Statement

Our project is **StaffAlloc**, an AI-powered staffing and hours allocation tool that helps project managers efficiently allocate employee hours across multiple projects while ensuring no one exceeds their funded hours or monthly capacity.


In [5]:
project_idea = """
StaffAlloc: AI-Powered Project Staffing & Hours Allocation Tool

A comprehensive staffing management application designed to help project managers efficiently allocate employee hours across multiple projects. 
Managers can create projects, add employees with their funded hours, allocate hours across project timelines based on 2-week sprints, 
and receive AI-powered recommendations to optimize resource allocation.

Core Features:
- Create and manage multiple projects with sprint-based timelines (2-week sprints)
- Add employees to projects with their Role, LCAT (Labor Category), Name, and total funded hours
- Allocate employee hours across project months/sprints with automatic FTE (Full-Time Equivalent) percentage calculation
- Define and manage full-time hours per month (automatically calculated based on business days, but editable by manager)
- Real-time validation to prevent over-allocation (funded hours and monthly capacity limits)
- Track both hours and FTE percentage for each employee per month
- Roll-up dashboard view showing all projects and employee allocations across the organization
- Calendar/timeline view showing employee availability across all projects
- RAG: Chat interface to query staffing ("How many hours does John have left?", "Who is available in March?", "What's the FTE for the Developer role in October?")
- Agent: AI-powered allocation recommendations based on employee role, LCAT, skills, availability, and project needs
- Agent: Conflict detection and resolution suggestions when employees are over-allocated
- Agent: Smart workload balancing across teams and projects
- Agent: Automatic FTE calculation and validation against standard thresholds

Example user flow:
A project manager creates an account
In the account, they can create multiple projects
For each project, they specify:
  - Project name, description, and timeline
  - Number of sprints (e.g., 18 sprints = 36 weeks = ~9 months)
  - Project phases or milestones
  - Full-time hours per month (system auto-calculates based on business days: e.g., September=168, October=176, November=144, etc., but manager can override)

Each employee has:
  - Role (e.g., Product Owner/Scrum Master, Developer, Cyber, SW Engineer)
  - LCAT (Labor Category for contracting/billing purposes)
  - Name
  - Total funded hours allocation (e.g., 160, 640, 1360 hours for the project duration)

The manager adds employees to projects and allocates their hours across the project timeline:
  - Hours can be split across months (e.g., 80 hours in September, 176 hours in October, 144 hours in November, etc.)
  - System automatically calculates FTE percentage (hours allocated / full-time hours for that month)
  - System displays FTE percentage alongside hours (e.g., 176 hours = 100% FTE, 88 hours = 50% FTE)
  - System validates that monthly allocations don't exceed the full-time hours for that month
  - System validates that total allocated hours don't exceed the employee's funded hours
  - Manager can see at a glance which months employees are at 100% FTE, 50% FTE, etc.

The manager gets a roll-up view showing:
  - All projects and their staffing levels
  - Each employee's total allocated hours vs. funded hours remaining
  - Visual indicators for over/under allocation
  - Timeline view showing FTE percentages across months
  - Breakdown by Role and LCAT for resource planning
  - Summary of how many sprints and weeks are in the project

AI agents help by:
  - Automatically calculating full-time hours per month based on business days
  - Computing FTE percentages in real-time as hours are allocated
  - Suggesting optimal hour distributions based on project phases and employee roles
  - Identifying scheduling conflicts and over-allocation scenarios
  - Recommending alternative staffing when constraints are detected
  - Predicting resource shortages before they occur
  - Balancing FTE loads across the team
 
The end result is an optimized staffing plan that maximizes resource utilization while respecting all constraints (funded hours, monthly capacity, FTE limits).
"""

problem_statement = """Project managers need an intelligent tool to allocate employee hours across multiple concurrent projects, 
tracking employee roles, LCATs (Labor Categories), and funded hours while automatically calculating FTE (Full-Time Equivalent) percentages. 
The tool must ensure no employee exceeds their funded hours or monthly capacity, provide roll-up visibility across all projects, 
handle variable full-time hours per month based on business days, and leverage AI to optimize resource allocation, 
calculate FTE percentages automatically, and identify conflicts before they become critical issues."""

print("Project Idea:")
print(project_idea)
print("\nProblem Statement:")
print(problem_statement)


Project Idea:

StaffAlloc: AI-Powered Project Staffing & Hours Allocation Tool

A comprehensive staffing management application designed to help project managers efficiently allocate employee hours across multiple projects. 
Managers can create projects, add employees with their funded hours, allocate hours across project timelines based on 2-week sprints, 
and receive AI-powered recommendations to optimize resource allocation.

Core Features:
- Create and manage multiple projects with sprint-based timelines (2-week sprints)
- Add employees to projects with their Role, LCAT (Labor Category), Name, and total funded hours
- Allocate employee hours across project months/sprints with automatic FTE (Full-Time Equivalent) percentage calculation
- Define and manage full-time hours per month (automatically calculated based on business days, but editable by manager)
- Real-time validation to prevent over-allocation (funded hours and monthly capacity limits)
- Track both hours and FTE percentage

## Step 3: Brainstorm Features and User Personas

Use the LLM to expand on our idea by brainstorming detailed features and identifying key user personas for the staffing allocation tool.


In [6]:
# Brainstorm features
features_prompt = f"""
Based on the following project idea, brainstorm a comprehensive list of features for this application.
Organize them into logical categories and provide detailed descriptions.

Project Idea:
{project_idea}

Problem Statement:
{problem_statement}

Context: Managers currently use Excel spreadsheets with the following structure:
- A row showing "Full Time Hours" per month (varies by month based on business days: e.g., 168, 176, 144, etc.)
- Employee rows with columns: Role, LCAT, Name, Funded Hours
- Monthly allocation columns showing both hours allocated and FTE percentage (e.g., "176" and "100%" or "88" and "50%")
- Project duration shown as number of sprints and weeks (e.g., "18 Sprints, 36 weeks")

Generate a detailed markdown list of features covering:
- Core staffing and hours allocation functionality with Role and LCAT tracking
- Automatic FTE percentage calculation (hours allocated / full-time hours for that month)
- Full-time hours per month management (auto-calculated based on business days, editable by manager)
- Project and employee management with role-based organization
- Time tracking and validation (funded hours, monthly capacity, FTE limits)
- Sprint-based timeline management (2-week sprints with automatic week/month calculations)
- Roll-up views and dashboards showing multi-project allocations
- Reporting and export capabilities (Excel-compatible formats)
- AI Agent features for optimization, conflict detection, FTE balancing, and recommendations
- RAG features for querying staffing data ("What's John's FTE in October?", "Show me all Developers")
- User experience enhancements for spreadsheet-like data entry

Pay special attention to features that improve on the Excel experience:
- Automatic FTE percentage calculations
- Real-time validation of over-allocation
- Cross-project visibility (not possible in separate Excel files)
- Smart suggestions based on Role and LCAT
- Automatic business day calculations for monthly hours
- Conflict detection when employee is allocated >100% FTE
- AI-powered optimization of hour distributions
- Natural language queries via RAG interface
"""

print("=" * 80)
print("BRAINSTORMING FEATURES")
print("=" * 80)
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)
print(brainstormed_features)
print("\n")


BRAINSTORMING FEATURES
Of course. Here is a comprehensive list of features for the **StaffAlloc** application, organized into logical categories with detailed descriptions.

---

### 1. Core Project Management

This category covers the foundational features for creating, defining, and managing projects within the system.

*   **Create & Manage Projects**
    *   **Description:** Users can create new projects by providing essential details like Project Name, Project Code/ID, Client, and a detailed Description. They can also edit, archive, or delete existing projects.
    *   **Fields:** Project Name, Code, Client, Description, Status (e.g., Planning, Active, On Hold, Completed).

*   **Sprint-Based Timelines**
    *   **Description:** Define project duration using a start date and the number of 2-week sprints. The system automatically calculates the end date, total weeks, and maps sprints to calendar months.
    *   **Example:** A manager enters "18 Sprints" and a start date. The system

In [7]:
# Identify user personas
personas_prompt = f"""
Based on the following project idea, identify three distinct user personas who would use this application.
For each persona, provide:
- A descriptive name
- Their background and role
- Their goals and motivations
- Their pain points when managing project staffing and employee hour allocation using Excel spreadsheets
- What they need from this tool

Project Idea:
{project_idea}

Problem Statement:
{problem_statement}

Context: Managers currently track staffing in Excel with:
- Columns for Role, LCAT (Labor Category), Name, Funded Hours
- Monthly allocations showing hours and FTE percentages
- Manual calculations and validation
- Separate spreadsheets for different projects (no cross-project visibility)
- Variable full-time hours per month based on business days

Consider different types of managers and users:
- Project managers allocating resources across multiple projects, tracking by Role and LCAT
- Senior managers/directors needing roll-up views across all projects with FTE summaries
- Resource managers focused on employee utilization, funded hours tracking, and capacity planning
- Contract managers who need LCAT-level reporting for billing purposes
- Individual employees who might need visibility into their allocations and remaining funded hours

Format your response as detailed markdown with clear sections for each persona.
"""

print("=" * 80)
print("IDENTIFYING USER PERSONAS")
print("=" * 80)
user_personas = get_completion(personas_prompt, client, model_name, api_provider)
print(user_personas)
print("\n")


IDENTIFYING USER PERSONAS
Of course. Based on the project idea for StaffAlloc, here are three distinct user personas who would use this application.

***

### Persona 1: The Pragmatic Project Manager

**Descriptive Name:** Priya, the Hands-On PM

*   **Background and Role:**
    Priya is a Senior Project Manager at a mid-sized government contracting firm. She has over 8 years of experience and is currently juggling two high-priority software development projects and one smaller cybersecurity assessment project. She is directly responsible for the delivery, timeline, and budget of these projects. Her teams are cross-functional, consisting of developers, cyber analysts, and scrum masters, each with specific LCATs and funded hour limits dictated by the contract.

*   **Goals and Motivations:**
    *   **Deliver on time and within budget:** Her primary goal is to ensure her projects are completed successfully without exceeding the allocated funds.
    *   **Maintain a balanced team:** She 

## Step 4: Generate Structured User Stories (JSON)

Transform the brainstormed features and personas into formal Agile user stories with acceptance criteria in Gherkin format, output as JSON.


In [8]:
json_user_stories_prompt = f"""
Act as a Senior Product Manager. Using the brainstormed features and user personas as context,
generate a comprehensive list of user stories for the StaffAlloc application.

Each user story must:
- Follow the format: "As a [persona], I want to [action], so that [benefit]"
- Include detailed acceptance criteria in Gherkin format (Given/When/Then)
- Be specific and actionable

Brainstormed Features:
{brainstormed_features}

User Personas:
{user_personas}

Output Requirements:
Your response MUST be a valid JSON array of objects. Each object must have these exact keys:
- "id": A unique identifier (e.g., "US001", "US002")
- "user_story": The user story text
- "persona": The persona name
- "acceptance_criteria": An array of strings with Gherkin-style criteria

Generate at least 15-20 user stories covering all major features, with special emphasis on:
- Adding employees with Role, LCAT, Name, and Funded Hours
- Allocating hours across months with automatic FTE percentage calculation
- Managing full-time hours per month (auto-calculated, but editable)
- Validating that monthly allocations don't exceed full-time hours
- Validating that total allocations don't exceed funded hours
- Sprint-based timeline management (e.g., 18 sprints = 36 weeks)
- AI agent features (automatic FTE calculations, recommendations, conflict detection, optimization)
- RAG query capabilities ("What's John's FTE in October?", "Show all Developers over 80% allocated")
- Roll-up views showing all projects with Role and LCAT breakdowns
- Cross-project visibility and conflict detection
- Excel-like data entry experience with real-time calculations

CRITICAL: Your response must begin with [ and end with ]. Do not include any text before or after the JSON array.
"""

print("=" * 80)
print("GENERATING USER STORIES AS JSON")
print("=" * 80)

json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Parse and validate the JSON
try:
    # Clean up markdown fences if present
    if '```' in json_output_str:
        json_output_str = json_output_str.split('```')[1].lstrip('json').strip()
    
    user_stories_json = json.loads(json_output_str)
    print(f"✓ Successfully parsed {len(user_stories_json)} user stories as JSON.\n")
    
    if user_stories_json:
        print("Sample User Story:")
        print(json.dumps(user_stories_json[0], indent=2))
    else:
        print("⚠ Warning: JSON array is empty.")

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"✗ Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str[:500])
    user_stories_json = []


GENERATING USER STORIES AS JSON
✓ Successfully parsed 20 user stories as JSON.

Sample User Story:
{
  "id": "US001",
  "user_story": "As a Priya, the Hands-On PM, I want to create a new project with a name, client, start date, and duration in sprints, so that I can establish the project's basic framework and timeline.",
  "persona": "Priya, the Hands-On PM",
  "acceptance_criteria": [
    "Given I am on the main project dashboard,",
    "When I click the 'Create New Project' button,",
    "Then a form appears with fields for Project Name, Code, Client, Start Date, and Number of Sprints.",
    "When I fill in the details and enter '18' for the number of sprints,",
    "And I click 'Save',",
    "Then the new project is created, its timeline is set to 36 weeks, and I am taken to its allocation grid."
  ]
}


## Step 5: Validate and Save User Stories

Programmatically validate the structure of our user stories and save them as a JSON artifact.


In [9]:
def validate_and_save_stories(stories_data, output_path="Artifacts/Documentation/user_stories.json"):
    """Validates the structure of the user stories data and saves it if valid."""
    if not isinstance(stories_data, list) or not stories_data:
        print("✗ Validation Failed: Data is not a non-empty list.")
        return False

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    for idx, story in enumerate(stories_data):
        # Check if the story is a dictionary
        if not isinstance(story, dict):
            print(f"✗ Validation Error: Story at index {idx} is not a dictionary.")
            all_stories_valid = False
            continue
        
        # Check if all required keys are present
        missing_keys = [key for key in required_keys if key not in story]
        if missing_keys:
            print(f"✗ Validation Error: Story at index {idx} (ID: {story.get('id', 'Unknown')}) is missing keys: {missing_keys}")
            all_stories_valid = False
            continue
        
        # Check if the acceptance_criteria list is not empty
        if not isinstance(story['acceptance_criteria'], list) or len(story['acceptance_criteria']) == 0:
            print(f"✗ Validation Error: Story at index {idx} (ID: {story['id']}) has empty or invalid acceptance_criteria.")
            all_stories_valid = False
            continue

    if all_stories_valid:
        print(f"✓ All {len(stories_data)} user stories passed validation.\n")
        
        # Save the artifact
        save_artifact(json.dumps(stories_data, indent=2), output_path)
        print(f"✓ User stories successfully saved to {output_path}")
        return True
    else:
        print("\n✗ Validation failed. Artifact not saved.")
        return False

# Validate and save
print("=" * 80)
print("VALIDATING AND SAVING USER STORIES")
print("=" * 80)

if user_stories_json:
    validation_success = validate_and_save_stories(user_stories_json)
else:
    print("⚠ Skipping validation as user_stories_json is empty or not defined.")
    validation_success = False


VALIDATING AND SAVING USER STORIES
✓ All 20 user stories passed validation.

✓ User stories successfully saved to Artifacts/Documentation/user_stories.json


## Step 6: Generate Product Requirements Document (PRD)

Now we'll use the validated user stories to generate a comprehensive PRD. We'll create a custom template structure for TripSplit.


In [15]:
# Define a PRD template structure for TripSplit
prd_template_structure = """
# Product Requirements Document Structure

## 1. Executive Summary & Vision
- Product name, overview, purpose, and vision

## 2. The Problem
- Problem statement
- User personas and scenarios

## 3. Goals & Success Metrics
- Table with Goal, KPI, and Target columns

## 4. Functional Requirements & User Stories
- Organized by Epic
- User stories with acceptance criteria

## 5. Non-Functional Requirements (NFRs)
- Performance
- Security
- Accessibility
- Scalability
- Usability
- Reliability

## 6. Technical Considerations
- Technology stack recommendations
- Database design considerations
- API design principles

## 7. Release Plan & Milestones
- MVP features
- Future versions

## 8. Out of Scope & Future Considerations
- Out of scope for V1.0
- Future work

## 9. Appendix & Open Questions
- Open questions
- Dependencies and assumptions
"""

prd_generation_prompt = f"""
You are a Senior Product Manager tasked with creating a comprehensive Product Requirements Document (PRD)
for StaffingAloc, a employee staffing tool for managers.

Project Context:
{project_idea}

User Stories:
{json.dumps(user_stories_json, indent=2)}

User Personas:
{user_personas}

Instructions:
1. Create a comprehensive PRD following this structure:
{prd_template_structure}

2. Use the provided user stories to populate the "Functional Requirements & User Stories" section
3. Group user stories into logical Epics
4. For sections not covered in user stories (Success Metrics, NFRs, Technical Considerations, Release Plan),
   use your expertise to create logical and appropriate content
5. Make the PRD specific to staffingAloc - avoid generic content
6. Ensure all sections are comprehensive and actionable
7. Format as clean, professional markdown

Generate the complete PRD now.
"""

print("=" * 80)
print("GENERATING PRODUCT REQUIREMENTS DOCUMENT")
print("=" * 80)

if user_stories_json:
    prd_output = get_completion(prd_generation_prompt, client, model_name, api_provider, temperature=0.3)
    print("✓ PRD generated successfully.\n")
    print("Preview (first 1000 characters):")
    print(prd_output)
    print("\n... (full document will be saved to file)")
    prd_path = "Artifacts/Documentation/prd.md"
    save_artifact(prd_output, prd_path, overwrite=True)
    print(f"✓ PRD successfully saved to {prd_path}")

else:
    print("⚠ Skipping PRD generation because user stories are missing.")
    prd_output = ""


GENERATING PRODUCT REQUIREMENTS DOCUMENT
✓ PRD generated successfully.

Preview (first 1000 characters):
Of course. Here is a comprehensive Product Requirements Document (PRD) for StaffAlloc, created based on the provided context and my expertise as a Senior Product Manager.

***

# Product Requirements Document: StaffAlloc
**Version:** 1.0
**Date:** October 26, 2023
**Author:** Senior Product Manager
**Status:** Draft

## 1. Executive Summary & Vision

### Product Name
StaffAlloc

### Overview
StaffAlloc is an intelligent, AI-powered staffing management application designed to replace cumbersome and error-prone spreadsheets. It provides project managers, directors, and resource managers with a centralized platform to create projects, allocate employee hours, track budgets, and optimize resource utilization across the entire organization.

### Purpose
The primary purpose of StaffAlloc is to solve the critical business challenges of manual resource planning. By automating calculations, 

## Step 7: Save the PRD

Save the generated PRD to our artifacts folder.


In [14]:
if prd_output:
    prd_path = "Artifacts/Documentation/prd.md"
    save_artifact(prd_output, prd_path, overwrite=True)
    print(f"✓ PRD successfully saved to {prd_path}")
else:
    print("⚠ No PRD to save.")


✓ PRD successfully saved to Artifacts/Documentation/prd.md


## Step 8: Generate Pydantic Validation Model

Create a Pydantic model to programmatically validate PRD structure in the future.


In [ ]:
pydantic_model_prompt = f"""
You are a Python developer creating a Pydantic model for validating Product Requirements Documents.

Based on this PRD structure:
{prd_template_structure}

Requirements:
1. Create a Python class named 'ProductRequirementsDocument' that inherits from Pydantic's BaseModel
2. Add fields for each major section of the PRD
3. Use appropriate Python types: str, List, Dict, Optional, etc.
4. For user stories, create a nested UserStory model with fields: id, persona, user_story, acceptance_criteria
5. For goals/metrics, create a nested GoalMetric model with fields: goal, kpi, target
6. Add descriptive docstrings with Field(..., description="...") for each field
7. Import all necessary modules (pydantic, typing, datetime if needed)
8. Make the model comprehensive and production-ready

Generate ONLY the Python code without markdown formatting. The code should be ready to save as a .py file.
"""

print("=" * 80)
print("GENERATING PYDANTIC VALIDATION MODEL")
print("=" * 80)

pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider, temperature=0.2)

# Clean up markdown fences if present
if '```' in pydantic_model_code:
    pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()

print("✓ Pydantic model generated.\n")
print("Preview (first 800 characters):")
print(pydantic_model_code[:800])
print("\n... (full model will be saved to file)")


## Step 9: Save the Pydantic Model

Save the validation model for future use.


In [ ]:
if pydantic_model_code:
    model_path = "Artifacts/Documentation/prd_validation_model.py"
    save_artifact(pydantic_model_code, model_path)
    print(f"✓ Pydantic model successfully saved to {model_path}")
else:
    print("⚠ No Pydantic model to save.")


## Summary: Phase 1 Complete! 🎉

We have successfully completed Phase 1 of the capstone project for **StaffAlloc**. Here's what we've generated:

### Artifacts Created:
1. **User Stories (JSON)**: `Artifacts/Documentation/user_stories.json`
   - Structured, validated user stories with acceptance criteria
   - Covers staffing allocation, Role/LCAT tracking, FTE calculations, AI features, and validation
   - Machine-readable format for downstream processing

2. **Product Requirements Document**: `Artifacts/Documentation/prd.md`
   - Comprehensive PRD with all major sections
   - Executive summary, problem statement, user personas
   - Functional requirements covering hours allocation, FTE calculations, sprint management
   - Non-functional requirements for performance, security, and scalability
   - AI agent and RAG capabilities for optimization and natural language queries
   - Release plan and technical considerations

3. **Pydantic Validation Model**: `Artifacts/Documentation/prd_validation_model.py`
   - Programmatic validation for PRD structure
   - Ensures consistency across documentation

### Key Features Documented:
- **Employee Management**: Role, LCAT, Name, Funded Hours tracking
- **Hours Allocation**: Monthly allocation with automatic FTE percentage calculation
- **Full-Time Hours**: Auto-calculated per month based on business days (editable)
- **Sprint-Based Timelines**: 2-week sprints with automatic week/month calculations
- **Validation**: Funded hours limits, monthly capacity checks, FTE threshold validation
- **Roll-up Views**: Multi-project dashboards with cross-project visibility
- **AI Agents**: Auto FTE calculation, conflict detection, optimization recommendations
- **RAG Interface**: Natural language queries for staffing data

### Next Steps:
- **Phase 2**: Use the PRD to generate system architecture and database schema
- **Phase 3**: Build the FastAPI backend with Role/LCAT/FTE models
- **Phase 4**: Create tests and perform security review
- **Phase 5**: Build the React frontend with spreadsheet-like UI

### Key Takeaways:
- LLMs can transform Excel-based workflows into structured, intelligent applications
- Detailed context (like the Excel structure) leads to more accurate requirements
- AI capabilities (agents and RAG) can significantly enhance traditional tools
- Template-driven generation with domain-specific prompts ensures relevance
